## 1. Import Stuff

In [ ]:
# !pip install tensorflow tensorflow-gpu matplotlib tensorflow-datasets ipywidgets

In [ ]:
# !pip list

In [ ]:
import tensorflow as tf
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [ ]:
# import tensorflow datasets for fashion mnist
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [ ]:
ds=tfds.load('fashion_mnist',split='train')

In [ ]:
ds.as_numpy_iterator().next().keys()

## 2. Visulaize Data


In [ ]:
import numpy as np 

In [ ]:
dataiterator=ds.as_numpy_iterator()

In [ ]:
# getting data out of pipeline
dataiterator.next()

In [ ]:
fig,ax=plt.subplots(ncols=4,figsize=(20,20))
for idx in range(4):
    sample=dataiterator.next()  # grab an image and label
    ax[idx].imshow(np.squeeze(sample['image']))
    ax[idx].title.set_text(sample['label'])

In [ ]:
np.squeeze(dataiterator.next()['image']).shape

In [ ]:
# scale and return images only
def scale_images(data):
    image=data['image']
    return image/255

In [ ]:
'''
steps in data pipeline :-
    map
    cache
    shuffle
    batch
    prefetch
    
    '''

In [ ]:
ds=tfds.load('fashion_mnist',split='train')  # reload data
ds=ds.map(scale_images) # map scale_images function on dataset
ds=ds.cache() # cache the dataset for that batch
ds=ds.shuffle(60000) # shuffle the data
ds=ds.batch(128) # batch into 128 images per sample
ds=ds.prefetch(64)  # store 64 images in cache

In [ ]:
ds.as_numpy_iterator().next().shape

## 3. Build Neural Network

In [ ]:
from tensorflow.keras.models import Sequential  # contain Gens and Degens
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Reshape,LeakyReLU,Dropout,UpSampling2D

### 3.1 Build Generator

In [ ]:
def build_generator():
    
    # takes in random values and reshapes it to 7*7*128
    # Beginnings of a generated image
    model=Sequential()
    model.add(Dense(7*7*128,input_dim=128)) # 7*7 is random value
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7,7,128)))
    
    # upsampling block 1
    model.add(UpSampling2D())
    model.add(Conv2D(128,5,padding='same'))
    model.add(LeakyReLU(0.2))
    
     # upsampling block 2
    model.add(UpSampling2D())
    model.add(Conv2D(128,5,padding='same'))
    model.add(LeakyReLU(0.2))
    
     # Conv block 1
    model.add(Conv2D(128,4,padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Conv block 2
    model.add(Conv2D(128,4,padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Conv layer to get one channel
    model.add(Conv2D(1,4,padding='same',activation='sigmoid'))
    
    
    return model

In [ ]:
generator=build_generator()

In [ ]:
generator.summary()

In [ ]:
img=generator.predict(np.random.randn(4,128,1)) # generate 4 random images with 128 random values
Img=img
fig,ax=plt.subplots(ncols=4,figsize=(20,20))
for idx,img in enumerate(img) :
    ax[idx].imshow(np.squeeze(img))
    ax[idx].title.set_text(idx)

### 3.2 Build Discriminator

In [ ]:
def build_discriminator():
    model=Sequential()
    
    # first Conv Block
    model.add(Conv2D(32,5,input_shape=(28,28,1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # second Conv Block
    model.add(Conv2D(64,5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
     # Third Conv Block
    model.add(Conv2D(128,5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
     # Fourth Conv Block
    model.add(Conv2D(256,5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    
    # flatten then pass to dense layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1,activation='sigmoid'))
    
    return model

In [ ]:
discriminator=build_discriminator()
discriminator.summary()

In [ ]:
Img

In [ ]:
img=Img[0]
img.shape

In [ ]:
discriminator.predict(np.expand_dims(img,0))

## 4. Training Loop

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
g_opt=Adam(learning_rate=0.0001)
d_opt=Adam(learning_rate=0.00001)
g_loss=BinaryCrossentropy()
d_loss=BinaryCrossentropy()

In [ ]:
from  tensorflow.keras.models import Model

In [ ]:
class FashionGAN(Model): 
    def __init__(self, generator, discriminator, *args, **kwargs):
        # Pass through args and kwargs to base class 
        super().__init__(*args, **kwargs)
        
        # Create attributes for gen and disc
        self.generator = generator 
        self.discriminator = discriminator 
        
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs): 
        # Compile with base class
        super().compile(*args, **kwargs)
        
        # Create attributes for losses and optimizers
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss 

    def train_step(self, batch):
        # Get the data 
        real_images = batch
        fake_images = self.generator(tf.random.normal((128, 128, 1)), training=False)
        
        # Train the discriminator
        with tf.GradientTape() as d_tape: 
            # Pass the real and fake images to the discriminator model
            yhat_real = self.discriminator(real_images, training=True) 
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            
            # Create labels for real and fakes images
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            
            # Add some noise to the TRUE outputs
            noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            
            # Calculate loss - BINARYCROSS 
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        # Apply backpropagation - nn learn 
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
        
        # Train the generator 
        with tf.GradientTape() as g_tape: 
            # Generate some new images
            gen_images = self.generator(tf.random.normal((128,128,1)), training=True)
                                        
            # Create the predicted labels
            predicted_labels = self.discriminator(gen_images, training=False)
                                        
            # Calculate loss - trick to training to fake out the discriminator
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
            
        # Apply backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        
        return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [ ]:
# Create instance of subclassed model
fashgan = FashionGAN(generator, discriminator)

In [ ]:
# Compile the model
fashgan.compile(g_opt, d_opt, g_loss, d_loss)

In [ ]:
import os
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

### 4.1 Build Callback

In [ ]:
      # Create a directory if it is not there, so we can save files and results in it
      from pathlib import Path
      Path('/kaggle/working/main_folder/sub_folder').mkdir(parents=True, exist_ok=True)


In [ ]:
class ModelMonitor(Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.uniform((self.num_img, self.latent_dim,1))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(os.path.join('/kaggle/working/main_folder/sub_folder', f'generated_img_{epoch}_{i}.png'))


## 5. Train

In [ ]:
# Recommend 2000 epochs
hist = fashgan.fit(ds, epochs=50, callbacks=[ModelMonitor()])

## 6. Review Performance

In [ ]:
plt.suptitle('Loss')
plt.plot(hist.history['d_loss'], label='d_loss')
plt.plot(hist.history['g_loss'], label='g_loss')
plt.legend()|
plt.show()

## 7. Test Generator

### 7.1 Generate images

In [ ]:
imgs = generator.predict(tf.random.normal((16, 128, 1)))

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=4, figsize=(10,10))
for r in range(4): 
    for c in range(4): 
        ax[r][c].imshow(imgs[(r+1)*(c+1)-1])

### 7.2 Save the Model

In [ ]:
generator.save('generator.h5')
discriminator.save('discriminator.h5')